In [2]:

import time, base64, os, re,json5, pprint,random , math, hashlib
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
import pandas as pd
import numpy as np
from io import StringIO

from app.operation_executor import OperationExecutor

In [3]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\cache_14082025_131119.json5"

data = Helper.load_json(path)

ops = OperationExecutor()
function_to_execute= {
    "sha256":"_generate_hash_sha256",
    "sha1":"_generate_hash_sha1",
}

processed_data = ops.runner(data,function_to_execute)

Helper.save_json(processed_data,"process.json")

Processing for Bank :Uco Bank
Processing for Bank :NSE Equity India


In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random
import time
import math


def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))


def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()


def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()


# ==== SETUP CHROME WITH GOOGLE PROFILE ====
profile_path = r"C:\Users\rando\AppData\Local\Google\Chrome\User Data"
profile_dir = "Profile 1" # or "Default"

options = uc.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

# ==== GO TO SITE ====
driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

# ==== SIMULATE HUMAN BEHAVIOR ====
human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()

import undetected_chromedriver as uc 

profile_path = r"C:\Users\kaustubh.keny\AppData\Local\Google\Chrome\User Data"
profile_dir = "Default" # Change to match your actual one

options = uc.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
# options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(driver_executable_path=r"C:\Users\kaustubh.keny\chromedriver-win64\chromedriver.exe",options=options)
driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

# import scrapy

# class UnionBankSpider(scrapy.Spider):
#     name = "unionbank_tables"
#     allowed_domains = ["unionbankofindia.co.in"]
#     start_urls = ["https://www.unionbankofindia.co.in/en/details/rate-of-interest"]

#     def parse(self, response):
#         tables = response.css("table").getall()
#         self.logger.info(f"Found {len(tables)} tables")
#         for idx, table in enumerate(tables):
#             yield {
#                 "table_index": idx,
#                 "html": table
#             }

